# <center>  W5 S2: Implementing Convolutional Neural Network for Image Classification


## Let's first load and preprocess our images
## Setup

In [ ]:
import numpy as np 
import os 
import PIL 
from PIL import Image 
import tensorflow as tf 
import tensorflow_datasets as tfds

# Import the flowers dataset
## From Add data button on the top right corner, import "flower-photos-by-the-tensorflow-team" dataset.
## This tutorial uses a dataset of several thousand photos of flowers. 
## The flowers dataset contains five sub-directories, one per class:

flowers_photos/

daisy/

dandelion/

roses/

sunflowers/

tulips/

In [ ]:
# path for the root dataset directory
data_dir = "../input/flower-photos-by-the-tensorflow-team/flower_photos"

## After downloading (218MB), you should now have a copy of the flower photos available. There are 3,670 total images:

In [ ]:
# let's see how many classes we have in our dataset
classes = os.listdir(data_dir)
classes.remove("LICENSE.txt")

# counting number of images per class
image_count = [len(os.listdir(os.path.join(data_dir,classes[i]))) for i in range(5)]

print(classes)
print(image_count)

## Before moving forward and building our super-cool TF model, let's look at the dataset a lil bit..

## Let's Google our dataset!

# Each directory contains images of that type of flower. Here are some roses:

In [ ]:
roses = os.listdir(os.path.join(data_dir, "roses"))

img = Image.open(os.path.join(data_dir, "roses", roses[8]))
img

# Load data using a Keras utility
## Let's load these images off our Kaggle kernel using the helpful **tf.keras.utils.image_dataset_from_directory** utility.
## Create a dataset, essentially!
## Start with defining some parameters for the loader:

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

## It's good practice to use a validation split when developing your model. You will use 80% of the images for training and 20% for validation.

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

## We used a random seed. What's that all about?!
Check here: https://towardsdatascience.com/how-to-use-random-seeds-effectively-54a4cd855a79

## You can find the class names in the class_names attribute on these datasets.

In [ ]:
class_names = train_ds.class_names
print(class_names)

# Visualize the data
## Here are the first nine images from the training dataset.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1): # Only take a single example
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

### You can train a model using these datasets by passing them to model.fit (shown later in this tutorial). 
### If you like, you can also manually iterate over the dataset and retrieve batches of images:

### How to get batches of images and labels? 


In [ ]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

### Why this works: https://keras.io/api/preprocessing/image/

### The image_batch is a tensor of the shape (32, 180, 180, 3). 
### This is a batch of 32 images of shape 180x180x3 (the last dimension refers to color channels RGB). 
### The label_batch is a tensor of the shape (32,), these are corresponding labels to the 32 images.
### You can call .numpy() on either of these tensors to convert them to a numpy.ndarray.

# Standardize the data
### The RGB channel values are in the [0, 255] range. This is not ideal for a neural network; in general you should seek to make your input values small.

### Here, you will standardize values to be in the [0, 1] range by using tf.keras.layers.Rescaling:

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Data augmentation
### It takes the approach of generating additional training data from your existing examples by augmenting them using random transformations that yield believable-looking images. 
### This helps expose the model to more aspects of the data and generalize better.

### You will implement data augmentation using the following Keras preprocessing layers: 
### **tf.keras.layers.RandomFlip**, **tf.keras.layers.RandomRotation**, and **tf.keras.layers.RandomZoom**. 
### These can be included inside your model like other layers, and run on the GPU.

In [ ]:
# add randomflip, randomrotation and randomzoom
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal", input_shape=(img_height,img_width,3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1)])

# Building our ML Model
## Now, you need to do some heavy lifting here. 
## For building our ML model, we will use tensorflow and keras, these libraries makes it very easy to apply ml with very little efforts

### Note: The below image is extracted from some other source and it may not exactly represent our model, 
### but it will give you a nice idea of a Convolutional Neural Network, something that we will be using here.

![cnn-11.png](attachment:4cfeae39-c406-4949-87f2-0bc4c853022b.png)

## So, as we have seen above, for our (180 X 180 X 3) image, our first layer of neural network will contain 97,200 units in one Dense Layer. And multiplications of all the Dense layers with each other is a lot of computation for our lil computers.

### That's where Convolutional layers comes into picture.

### The following animation shows a convolutional layer consisting of 9 convolutional operations involving the 5x5 input matrix. Notice that each convolutional operation works on a different 3x3 slice of the input matrix. The resulting 3x3 matrix (on the right) consists of the results of the 9 convolutional operations:

![](http://developers.google.com/machine-learning/glossary/images/AnimatedConvolution.gif)

# Convolutional Neural Network
## A neural network in which at least one layer is a convolutional layer. A typical convolutional neural network consists of some combination of the following layers:

### convolutional layers
### pooling layers
### dense layers

## Convolutional neural networks have had great success in certain kinds of problems, such as image recognition.

# Pooling

![](http://developers.google.com/machine-learning/glossary/images/PoolingConvolution.svg) 

## Here are some components to enhance the performance of our model

## Dropout - leave out some neurons at random!

## BatchNormalization - [https://towardsdatascience.com/batch-norm-explained-visually-how-it-works-and-why-neural-networks-need-it-b18919692739](http://)

# Train a model
## For completeness, you will show how to train a simple model using the datasets you have just prepared.

## The Sequential model consists of three convolution blocks (tf.keras.layers.Conv2D) with a max pooling layer (tf.keras.layers.MaxPooling2D) in each of them. 

## There's a fully-connected layer (tf.keras.layers.Dense) with 128 units on top of it that is activated by a ReLU activation function ('relu').

In [ ]:
num_classes = 5

# Sequential groups a linear stack of layers into a tf.keras.Model.
model = tf.keras.Sequential([
    
    data_augmentation,
    tf.keras.layers.Rescaling(1./255),

    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D  
    tf.keras.layers.Conv2D(32, 3, activation='relu'),

    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D  
    tf.keras.layers.MaxPooling2D(),

    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
    tf.keras.layers.Dropout(0.2),
    
    # https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization
    tf.keras.layers.BatchNormalization(),
    
    
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])

## Choose the tf.keras.optimizers.Adam optimizer and tf.keras.losses.SparseCategoricalCrossentropy loss function. 

## To view training and validation accuracy for each training epoch, pass the metrics argument to Model.compile.

In [ ]:
# Configures the model for training.

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

## Note: You will only train for a few epochs so this tutorial runs quickly - or use GPU feature: Accelerator!

In [ ]:
# remember to turn on the GPU from notebook settings
# if you don't see the Accelarator option on the right hand side, go to your kaggle profile and verify your profile by adding your mobile number

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

# it should take 30-40 seconds to train our model for first 3 epochs. And the val_accuracy should be around 0.46

## Note: You can also write a custom training loop instead of using Model.fit. 
## To learn more, visit the [Writing a training loop from scratch](https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch) tutorial.

# Visualize training results
## Create plots of loss and accuracy on the training and validation sets:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


# Next week you will get to learn -

## Transfer Learning
## Data Annotation
## Handling Images with OpenCV

# Time for some practice questions!

Q1.  Apply CNN for Image Classification.
1. Download the notebook and import it to the Kaggle.

2. From Add data button on the top right corner, import cars-image-dataset  
  Find the dataset with this link in the Kaggle : ../input/cars-image-dataset


Now first let's explore the dataset. Figure out the different classes present. Also see the training and testing data.

a. Show the images of class **swift**.

b. Load the dataset from keras utils and check if there any need for splitting dataset.

c. After loading Visualize the data.

d. Perform the necessary preprocessing required in the dataset(can include standardization, Normalization, data augmentation etc.) which you think are necessary to perform.  

e. Train the model add layers according to you choice.

f. Figure out validation accuracy for testing data and Visualize the results.



# Thank you completing the notebook!